In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Experiments

#### Imports

In [ ]:
import numpy as np
import torch
import time
import pytorch_lightning as pl
from pytorch_lightning import loggers
import config
from utils.Visual import draw_plot

# 1. Hyperparameter Tuning

*see coresponding notebook!*

# 2. Adversarial Training

In [ ]:
##############################################################################################################
##############################################################################################################
##############################################################################################################
from utils.TrainHelper import TrainHelper, get_all_models_data
from attacks.pgd import ProjectedGradientDescent
from classification.models.M5 import M5PLModule

# define hyperparameters
hparams = {
    "batch_size": 64,
    "learning_rate": 3e-4,
    "weight_decay": 0.1,
    "lr_decay": 0.95,
    "epochs": 20
}

# define attack configs
attack_configs = [
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 5
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_2, eps=5"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 10
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_2, eps=10"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 20
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_2, eps=20"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 50
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_2, eps=50"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 100
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_2, eps=100"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 200
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_2, eps=200"}
                   },
    
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 500
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_1, eps=500"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 1000
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_1, eps=1000"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 2000
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_1, eps=2000"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 5000
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_1, eps=5000"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 10000
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_1, eps=10000"}
                   },
    
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.05
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_inf, eps=0.05"}
                   },
    
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.1
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_inf, eps=0.1"}
                   },
    
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.2
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_inf, eps=0.2"}
                   },
    
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.3
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_inf, eps=0.3"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.4
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_inf, eps=0.4"}
                   },

                  ]

th = TrainHelper()
th.run(M5PLModule, hparams, attack_configs, save_epochs = [100])
##############################################################################################################
##############################################################################################################
##############################################################################################################

In [ ]:
from utils.TrainHelper import TrainHelper, get_all_models_data, get_model_info

list(get_all_models_data()["M5"].keys())

sorted([ (m["final_val_acc"], m["path"]) for m in get_all_models_data()["M5"].values()])[-5:]

In [ ]:
print(get_model_info("M5_attack_ProjectedGradientDescent_v88.p"))

In [ ]:
sorted([ m["path"] for m in get_all_models_data()["M5"].values()])[-5:]

# 3. Robustness Comparison

In [ ]:
##############################################################################################################
##############################################################################################################
##############################################################################################################

from utils.RobustnessExperiment import RobustnessExperiment, load_experiment
from attacks.FGA_Batch import fast_gradient_attack
from attacks.pgd import ProjectedGradientDescent
from classification.models.M5 import M5PLModule

exp_config = [{
              "attack_fn": ProjectedGradientDescent, 
              "attack_arg": {"norm":["inf"], "epsilon": [0, 0.0001, 0.0002, 0.0005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1]},
              "meta": {"key_result":"success_rate", "key_config":"epsilon", "title":"FGSM"}
             },
            {
            "attack_fn": ProjectedGradientDescent, 
            "attack_arg": {"norm":["2"], "epsilon": [0, 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]},
            "meta": {"key_result":"success_rate", "key_config":"epsilon", "title":"FGA L2"}
            },
            {
            "attack_fn": ProjectedGradientDescent, 
            "attack_arg": {"norm":["1"], "epsilon": [100, 200, 500, 1000, 2000, 5000, 1e4, 2e4, 5e4, 1e5]},
            "meta": {"key_result":"success_rate", "key_config":"epsilon", "title":"FGA L1"}
            }]

# uncomment this if you want to create a new experiment instead!
#experiment = RobustnessExperiment(exp_config, title="EVALUATION_STANDARD")
experiment = load_experiment(exp_folder = "0001_EVALUATION_STANDARD")

#experiment.run("./1592000682.p", M5PLModule) #20 epochs normal training with hparams above
experiment.run("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_attack_ProjectedGradientDescent_v8_epoch_25.p", M5PLModule) #5 epochs adv training with hparams above

models = ['/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_attack_L_inf, iters=1, eps=0.1, w_dec=0.1_v123.p', '/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_attack_L_inf, iters=2, eps=0.1, w_dec=0.1_v124.p']
for m in models:
    experiment.run(m, M5PLModule)
##############################################################################################################
##############################################################################################################
##############################################################################################################

# Further Analyzis

### Show all evaluated models

In [ ]:
experiment.show_evaluated_models()

### Which Model performs best on Robustness-Evaluation?

In [ ]:
experiment.show_best_models(metric = "success_rate", best_n=1, limit_eps=8)

### Print info of best model

In [ ]:
from utils.TrainHelper import TrainHelper, get_all_models_data

get_all_models_data()["M5"]["M5_attack_try_regularization_v94.p"]

### Compare

In [ ]:
experiment.compare(config_key = "epsilon", results_key = "success_rate", models=['vanilla_model.p',
                                                                                 "M5_attack_ProjectedGradientDescent_v13.p",
                                                                                 "M5_attack_ProjectedGradientDescent_v91.p",
                                                                                 "M5_attack_ProjectedGradientDescent_v89.p",
                                                                                 "M5_attack_ProjectedGradientDescent_v93.p",
                                                                                 "M5_attack_try_regularization_v94.p",
                                                                                 "M5_attack_try_regularization_v96.p"
                                                                                ])

In [ ]:
experiment.compare(config_key = "epsilon", results_key = "success_rate", models=['1592000682.p',
                                                                                 "M5_attack_ProjectedGradientDescent_v69_epoch_10.p",
                                                                                 "M5_attack_ProjectedGradientDescent_v69_epoch_20.p",
                                                                                 "M5_attack_ProjectedGradientDescent_v69_epoch_30.p",
                                                                                 "M5_attack_ProjectedGradientDescent_v69_epoch_40.p",
                                                                                 "M5_attack_ProjectedGradientDescent_v69.p",
                                                                                 "M5_attack_ProjectedGradientDescent_v89.p"

                                                                                ])

In [ ]:
!du -sh *
!cd && du -sh 
!rm -rf checkpoints

# Results

Hyperparameters, if not mentioned explicitely, have been:

```python
hparams = {
    "batch_size": 64,
    "learning_rate": 3e-4,
    "weight_decay": 0.001,
    "lr_decay": 0.95,
    "epochs": 20
}
```


### Differerent fixed `epsilon`
We start by evaluating different fixed values for `epsilon`, as epsilon is the value that most significantly impacts the attack behavior and using a fixed value is the simplest and most meaningful sampling mode.

In [ ]:
attack_configs = [
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 5
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_2, eps=5"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 10
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_2, eps=10"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 20
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_2, eps=20"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 50
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_2, eps=50"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 100
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_2, eps=100"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 200
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_2, eps=200"}
                   },
    
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 500
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_1, eps=500"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 1000
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_1, eps=1000"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 2000
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_1, eps=2000"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 5000
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_1, eps=5000"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 10000
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_1, eps=10000"}
                   },
    
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.05
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_inf, eps=0.05"}
                   },
    
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.1
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_inf, eps=0.1"}
                   },
    
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.2
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_inf, eps=0.2"}
                   },
    
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.3
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_inf, eps=0.3"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.4
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_inf, eps=0.4"}
                   },

### Different sampling modes

### Different `num_iter`:

* Red: Baseline (no adversarial training).
* Blue: **1** iteration
* Green: **2** iterations
* Yellow: **3** iterations
* Bright Blue: **4** iterations
* Purple: **4** iterations, but epsilon = 0.05
* Black: **2** itertions, sampling logarithmically

**Conclusion:**

* **increasing the number of iterations brings significant improvements in robustness (i.e., lower success rate)**
* while trained with FGSM, the robustness also improves for FGA with L1 and L2 norm


In [ ]:
experiment.compare(config_key = "epsilon", results_key = "success_rate", models=['vanilla_model.p',
                                                                             ('M5_attack_num_iters_v97.p', "1 iteration"),
                                                                             ('M5_attack_num_iters_v98.p', "2 iterations"),
                                                                             ('M5_attack_num_iters_v99.p', "3 iterations"),
                                                                             ('M5_attack_num_iters_v100.p', "4 iterations"),
                                                                             ('M5_attack_num_iters_v101.p', "4 iterations, eps=0.05")]
                                                                            )

In [ ]:
experiment.compare(config_key = "epsilon", results_key = "success_rate", models=['1592000682.p',
                                                                              'M5_attack_num_iters_l2_v103.p',
 'M5_attack_num_iters_l2_v104.p',
 'M5_attack_num_iters_l2_v105.p',
 'M5_attack_num_iters_l2_v106.p',
 'M5_attack_num_iters_l2_v107.p',
 'M5_attack_num_iters_l2_v108.p']
                                                                            )

### Different `weight_decay`:

### Different number of `epochs`:

### Different `norms`:

In [ ]:
    {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.1
                                     ], "num_iter": [2]
                                 },
                   "META": {"TITLE": "num_iters"}
                   },
                      {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.1
                                     ], "num_iter": [3]
                                 },
                   "META": {"TITLE": "num_iters"}
                   },
                      {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.1
                                     ], "num_iter": [4]
                                 },
                   "META": {"TITLE": "num_iters"}
                   },
                          {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ 0.05
                                     ], "num_iter": [4]
                                 },
                   "META": {"TITLE": "num_iters"}
                   },
                          {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[ {"SAMPLING_MODE": "log", "l": 0.01, "u": 0.2}
                                     ], "num_iter": [2]
                                 },
                   "META": {"TITLE": "num_iters,sample"}
                   },
                              {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 10
                                     ], "num_iter": [2]
                                 },
                   "META": {"TITLE": "num_iters_l2"}
                   },
                                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 5
                                     ], "num_iter": [2]
                                 },
                   "META": {"TITLE": "num_iters_l2"}
                   },
                                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 7
                                     ], "num_iter": [3]
                                 },
                   "META": {"TITLE": "num_iters_l2"}
                   },
                                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ 5
                                     ], "num_iter": [4]
                                 },
                   "META": {"TITLE": "num_iters_l2"}
                   },
    
                                    {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[{ "SAMPLING_MODE": "log", "l": 1, "u": 20}
                                     ], "num_iter": [2]
                                 },
                   "META": {"TITLE": "num_iters_l2"}
                   },
    
                                    {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ {"SAMPLING_MODE": "log", "l": 1, "u": 10}
                                     ], "num_iter": [3]
                                 },
                   "META": {"TITLE": "num_iters_l2"}
                   },
    
                                    {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["2"], 
                                   "epsilon":[ {"SAMPLING_MODE": "log", "l": 1, "u": 10}
                                     ], "num_iter": [4]
                                 },
                   "META": {"TITLE": "num_iters_l2"}
                   },
                                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 5000
                                     ], "num_iter": [1]
                                 },
                   "META": {"TITLE": "num_iters_l1"}
                   },
    
                                      {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 5000
                                     ], "num_iter": [2]
                                 },
                   "META": {"TITLE": "num_iters_l1"}
                   },
                                      {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 2000
                                     ], "num_iter": [3]
                                 },
                   "META": {"TITLE": "num_iters_l1"}
                   },
                                      {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["1"], 
                                   "epsilon":[ 1000
                                     ], "num_iter": [4]
                                 },
                   "META": {"TITLE": "num_iters_l1"}
                   }